# Bio-inspired Optimization Algorithms - Demo

This notebook demonstrates how to use the `bioopt_bench` package for running optimization benchmarks.

In [ ]:
# Install the package if not already installed
# !pip install -e ..

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from bioopt_bench.algorithms import (
    GeneticAlgorithm,
    ParticleSwarmOptimization,
    AntColonyOptimization,
    GreyWolfOptimizer,
)
from bioopt_bench.tasks import (
    BealeFunction,
    EasomFunction,
    RastriginFunction,
    TSPTask,
    SchedulingTask,
)
from bioopt_bench.benchmark import run_one, run_suite
from bioopt_bench.registry import list_available

## 1. List Available Components

In [ ]:
available = list_available()
print("Algorithms:", available["algorithms"])
print("\nTasks:")
for task_type, variants in available["tasks"].items():
    print(f"  {task_type}: {variants}")
print("\nSuites:", available["suites"])

## 2. Benchmark Functions

### 2.1 Beale Function Optimization

In [ ]:
# Create task and algorithm
beale = BealeFunction()
pso = ParticleSwarmOptimization(n_particles=50)

# Run optimization
solution, history, meta = pso.run(beale, seed=42, iterations=200)

print(f"Best solution: {solution}")
print(f"Best fitness: {history['best_fitness'][-1]:.6f}")
print(f"Expected optimum: [3.0, 0.5] with f(x) = 0")

In [ ]:
# Plot convergence
plt.figure(figsize=(10, 5))
plt.plot(history["iter"], history["best_fitness"], "b-", label="Best Fitness")
plt.plot(history["iter"], history["mean_fitness"], "r--", alpha=0.7, label="Mean Fitness")
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("PSO on Beale Function")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 2.2 Compare GA vs PSO on Easom Function

In [ ]:
easom = EasomFunction()

# Run GA
ga = GeneticAlgorithm(population_size=100, num_bits=12)
ga_solution, ga_history, _ = ga.run(easom, seed=42, iterations=200)

# Run PSO
pso = ParticleSwarmOptimization(n_particles=50)
pso_solution, pso_history, _ = pso.run(easom, seed=42, iterations=200)

print(f"GA best: {ga_solution} -> {ga_history['best_fitness'][-1]:.6f}")
print(f"PSO best: {pso_solution} -> {pso_history['best_fitness'][-1]:.6f}")
print(f"Expected: [pi, pi] -> -1.0")

In [ ]:
# Compare convergence
plt.figure(figsize=(10, 5))
plt.plot(ga_history["iter"], ga_history["best_fitness"], "b-", label="GA")
plt.plot(pso_history["iter"], pso_history["best_fitness"], "r-", label="PSO")
plt.xlabel("Iteration")
plt.ylabel("Best Fitness")
plt.title("GA vs PSO on Easom Function")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 2.3 Rastrigin Function (Higher Dimension)

In [ ]:
rastrigin = RastriginFunction(n_dims=10)
pso = ParticleSwarmOptimization(n_particles=50)

solution, history, meta = pso.run(rastrigin, seed=42, iterations=300)

print(f"Best fitness: {history['best_fitness'][-1]:.4f}")
print(f"Solution: {[f'{x:.4f}' for x in solution]}")

## 3. Travelling Salesman Problem (TSP)

### 3.1 Random Graph with ACO

In [ ]:
# Create TSP task
tsp = TSPTask(n_cities=30, graph_type="random", seed=42)

# Run ACO
aco = AntColonyOptimization(n_ants=30)
tour, history, meta = aco.run(tsp, seed=42, iterations=100)

print(f"Best tour distance: {history['best_fitness'][-1]:.2f}")
print(f"Tour: {tour[:10]}... (first 10 cities)")

In [ ]:
# Visualize the tour
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot tour
ax = axes[0]
nodes = tsp.nodes
x = [nodes[i][0] for i in tour] + [nodes[tour[0]][0]]
y = [nodes[i][1] for i in tour] + [nodes[tour[0]][1]]
ax.plot(x, y, "b-o", markersize=6)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_title(f"Best Tour (Distance: {history['best_fitness'][-1]:.2f})")
ax.grid(True, alpha=0.3)

# Plot convergence
ax = axes[1]
ax.plot(history["iter"], history["best_fitness"], "b-", label="Best")
ax.plot(history["iter"], history["mean_fitness"], "r--", alpha=0.7, label="Mean")
ax.set_xlabel("Iteration")
ax.set_ylabel("Tour Distance")
ax.set_title("ACO Convergence")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.2 Compare GA vs ACO on TSP

In [ ]:
tsp = TSPTask(n_cities=25, graph_type="grid")

# GA
ga = GeneticAlgorithm(population_size=100, mutation_rate=0.05)
ga_tour, ga_history, _ = ga.run(tsp, seed=42, iterations=200)

# ACO
aco = AntColonyOptimization(n_ants=50)
aco_tour, aco_history, _ = aco.run(tsp, seed=42, iterations=100)

print(f"GA best distance: {ga_history['best_fitness'][-1]:.2f}")
print(f"ACO best distance: {aco_history['best_fitness'][-1]:.2f}")

## 4. Class Scheduling with GWO

In [ ]:
# Create scheduling task
scheduling = SchedulingTask()

print(f"Courses: {scheduling.n_courses}")
print(f"Time slots: {scheduling.n_time_slots}")
print(f"Classrooms: {scheduling.n_classrooms}")

In [ ]:
# Run GWO
gwo = GreyWolfOptimizer(n_wolves=100)
solution, history, meta = gwo.run(scheduling, seed=42, iterations=300)

print(f"Best fitness (penalty): {history['best_fitness'][-1]}")
print(f"Conflicts: {meta.get('conflicts_count', 'N/A')}")

In [ ]:
# Show schedule
print("\nOptimized Schedule:")
print("-" * 80)
formatted = scheduling.format_schedule(solution)
for entry in formatted:
    print(f"{entry['course']:25} | {entry['professor']:15} | {entry['time_slot']} | {entry['classroom']}")

In [ ]:
# Plot fitness evolution
plt.figure(figsize=(10, 5))
plt.plot(history["iter"], history["best_fitness"], "b-", label="Best Fitness")
plt.xlabel("Iteration")
plt.ylabel("Fitness (Penalty)")
plt.title("GWO Scheduling Optimization")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Using the Benchmark Runner

In [ ]:
# Run a single benchmark with full artifact saving
result = run_one(
    task_type="functions",
    task_variant="beale",
    algo_name="pso",
    iterations=100,
    seed=42,
    save=False,  # Set to True to save artifacts
)

print(f"Run ID: {result.run_id}")
print(f"Best fitness: {result.metrics.best_fitness:.6f}")
print(f"Runtime: {result.metrics.runtime_s:.3f}s")

In [ ]:
# Run a quick suite
results = run_suite(
    suite_name="quick",
    repeat=1,
    base_seed=42,
    save=False,
    verbose=True,
)

In [ ]:
# Summarize results
print("\nSuite Results Summary:")
print("-" * 60)
for r in results:
    print(f"{r.algorithm.name:5} on {r.task.variant:12} -> {r.metrics.best_fitness:12.4f} ({r.metrics.runtime_s:.2f}s)")

## 6. CLI Commands

You can also run benchmarks from the command line:

```bash
# List available components
python -m bioopt_bench list

# Run a single benchmark
python -m bioopt_bench run --task functions --fn beale --algo pso --iters 200 --seed 42 --save

# Run a suite
python -m bioopt_bench suite --suite default --repeat 3 --save
```